## Multiple Chain 快速入门

Runnables 可以轻松地用来串联多个 Chains，使用 RunnablePassthrough 将输出同时传给多条后继链。

```
     Input
      / \
     /   \
 Chain1 Chain2
     \   /
      \ /
      Combine
```

本指南展示如何使用 Runnable 实现多个 AI 关于相同话题的辩论：

```
    输入话题
       |
       |
    原始观点
      / |\
     /  |  \
 正面论述| 反面论述
     \  | /
      \ |/
     最终总结
```

In [1]:
# 导入相关模块，包括运算符、输出解析器、聊天模板、ChatOpenAI 和 运行器
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI,AzureChatOpenAI
from langchain_core.runnables import RunnablePassthrough
import os

openai_enable = True
if os.getenv('AZURE_OPENAI_ENDPOINT'):
    print('Azure mode')
    openai_enable = False
else:
    print('OpenAI mode')  

if openai_enable:
    llm = ChatOpenAI(model="gpt-4o-mini")
else:
    llm = AzureChatOpenAI(
        azure_deployment="checkGPT35_16K",
        api_version="2024-05-01-preview",
        temperature=0,
        max_tokens=None,
        timeout=None,
        max_retries=2
    )

# 创建一个计划器，生成一个关于给定输入的论证
planner = (
    ChatPromptTemplate.from_template("生成关于以下内容的论点: {input}")
    | llm
    | StrOutputParser()
    | {"base_response": RunnablePassthrough()}
)

# 创建正面论证的处理链，列出关于基础回应的正面或有利的方面
arguments_for = (
    ChatPromptTemplate.from_template(
        "列出关于{base_response}的正面或有利的方面"
    )
    | llm
    | StrOutputParser()
)

# 创建反面论证的处理链，列出关于基础回应的反面或不利的方面
arguments_against = (
    ChatPromptTemplate.from_template(
        "列出关于{base_response}的反面或不利的方面"
    )
    | llm
    | StrOutputParser()
)

# 创建最终响应者，综合原始回应和正反论点生成最终的回应
final_responder = (
    ChatPromptTemplate.from_messages(
        [
            ("ai", "{original_response}"),
            ("human", "正面观点:\n{results_1}\n\n反面观点:\n{results_2}"),
            ("system", "给出批评后生成最终回应"),
        ]
    )
    | llm
    | StrOutputParser()
)

# 构建完整的处理链，从生成论点到列出正反论点，再到生成最终回应
chain = (
    planner
    | {
        "results_1": arguments_for,
        "results_2": arguments_against,
        "original_response": itemgetter("base_response"),
    }
    | final_responder
)

Azure mode


In [3]:
print(chain.invoke({"input": "房地产低迷"}))

尽管房地产低迷可能会带来一些负面影响，但我们也要看到其中的积极因素。例如，房地产调控政策的出台可以稳定市场和促进经济发展，降低房地产泡沫风险，优化资源配置，促进租赁市场发展，降低房价对居民的负担等。这些措施可以帮助我们应对房地产低迷带来的挑战，并为经济发展创造更好的环境。因此，我们应该采取相应的政策和措施来平衡房地产市场，促进经济的稳定和发展。


In [4]:
print(chain.invoke({"input": "参加课外班太多对孩子成长不力"}))

尽管参加适量的课外班可以带来一些好处，但我们也要认识到过多的课外班可能会对孩子的成长产生负面影响。我们应该在安排孩子的课外活动时，要适度平衡，确保孩子能够充分发展和成长。

对于正面观点中提到的参加课外班可以提供更多学习机会和知识广度的观点，我们可以考虑通过其他方式来拓宽孩子的知识面，例如阅读、参观博物馆、参加社区活动等。这些活动可以提供丰富的学习机会，同时也给予孩子足够的自由时间和休息时间。

关于参加课外班可以培养孩子的自律和时间管理能力的观点，我们可以鼓励孩子参与一些自主学习的活动，例如制定学习计划、完成个人项目等，以培养他们的自律性和时间管理能力。

对于参加课外班可以提高孩子的竞争力和综合素质的观点，我们可以考虑通过其他方式来培养孩子的综合素质，例如参加社团活动、志愿者工作等。这些活动可以提供丰富的社交和领导经验，同时也给予孩子足够的时间进行自主学习和探索。

最重要的是，我们要尊重孩子的个体差异和兴趣发展，不要过于功利化地安排他们的课外活动。我们应该鼓励孩子发现和追求自己真正的兴趣，并给予他们足够的时间和空间去发展和探索。

总之，虽然参加适量的课外班可以带来一些好处，但过多的课外班可能会对孩子的成长产生负面影响。我们应该在安排孩子的课外活动时，要适度平衡，确保孩子能够充分发展和成长。


#### 流式输出

In [5]:
## chain 最终输出经过了 StrOutputParser 处理，所以可以直接输出流式输出 s
for s in chain.stream({"input": "全球经济"}):
    print(s, end="", flush=True)

尽管全球经济发展确实存在一些负面影响，但我们不能忽视其带来的积极影响。全球经济的创新和技术进步为社会带来了巨大的进步和福祉的提高。科学研究和技术发展的推动力来自于全球经济的发展，这使得我们能够更好地解决全球性问题，如医疗保健、食品安全和环境保护等。此外，全球经济的发展也为国际合作和和平稳定提供了机会。通过经济联系和相互依赖，各国更有动力通过对话和合作解决争端，减少战争和冲突的可能性。

当然，我们也要认识到全球经济发展中存在的问题，并采取措施来解决这些问题。例如，我们可以通过可持续发展的方式来平衡经济增长和环境保护，推动绿色技术和可再生能源的发展。我们也可以通过改革国际经济秩序，确保更加公平和包容的全球经济发展。此外，我们应该加强劳工权益的保护，确保劳工在全球经济中能够获得公平的待遇和工作条件。

综上所述，全球经济的发展既有积极的一面，也有负面的一面。我们应该充分认识到这些问题，并采取措施来解决和缓解其负面影响，以实现更加可持续和包容的全球经济发展。

### Homework: 实现一个多链版本的代码生成，输入功能需求，输出2种（Python，Java）以上编程语言的代码实现。

In [19]:
os.environ['LANGCHAIN_TRACING_V2']='true'
os.environ['LANGCHAIN_ENDPOINT']="https://api.smith.langchain.com"
os.environ['LANGCHAIN_API_KEY']="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"
os.environ['LANGCHAIN_PROJECT']="zz-testchain-1"

In [20]:
planner = (
    ChatPromptTemplate.from_template('生成关于以下功能需求的Python代码: {requirement}, 记得首先陈述一下需求是什么')
    | llm 
    | StrOutputParser()
    | {"python_code": RunnablePassthrough()}
)

#print(planner.invoke({'requirement': '冒泡排序'}))

In [14]:
Java_gen = (
    ChatPromptTemplate.from_template("用Java把同样的功能再实现一遍: {python_code}")
    | llm
    | StrOutputParser()
)
CPP_gen = (
    ChatPromptTemplate.from_template("用C++把同样的功能再实现一遍: {python_code}")
    | llm
    | StrOutputParser()
)


In [15]:
chain = (
    planner
    | {
        "results1": itemgetter('python_code'),
        'results2': Java_gen,
        'results3': CPP_gen,
        #'question': itemgetter('requirement')
        }
    | (lambda x: f"""
----------------------------- Python 实现:
{x['results1']}

----------------------------- Java 实现:
{x['results2']}

----------------------------- C++ 实现:
{x['results3']}
""")
)

In [21]:
print(chain.invoke({'requirement':'quick sorting'}))


----------------------------- Python 实现:
需求：实现快速排序算法。

代码实现如下：

```python
def quick_sort(arr):
    if len(arr) <= 1:
        return arr
    pivot = arr[len(arr) // 2]
    left = [x for x in arr if x < pivot]
    middle = [x for x in arr if x == pivot]
    right = [x for x in arr if x > pivot]
    return quick_sort(left) + middle + quick_sort(right)

# 示例用法
arr = [3, 1, 5, 2, 4]
sorted_arr = quick_sort(arr)
print(sorted_arr)
```

以上代码实现了快速排序算法。首先，定义了一个名为`quick_sort`的函数，该函数接受一个列表作为输入，并返回排序后的列表。在函数内部，首先判断列表长度是否小于等于1，如果是，则直接返回该列表。然后，选择列表中间位置的元素作为基准值（pivot）。接下来，将列表分成三部分：小于基准值的元素、等于基准值的元素和大于基准值的元素。使用列表推导式将元素分别放入左、中、右三个列表中。最后，通过递归调用`quick_sort`函数对左右两个列表进行排序，并将结果与中间列表合并后返回。

示例用法展示了如何使用该函数对一个列表进行排序，并打印排序后的结果。

----------------------------- Java 实现:
以下是使用Java实现的快速排序算法：

```java
import java.util.Arrays;

public class QuickSort {
    public static void main(String[] args) {
        int[] arr = {3, 1, 5, 2, 4};
        quickSort(arr, 0, arr.length - 1);
        System.out.println(Arrays.toString(

In [18]:
LANGCHAIN_PROJECT="pr-candid-collard-51"
llm.invoke('hello, world')

AIMessage(content='Hello! How can I assist you today?', response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 10, 'total_tokens': 19}, 'model_name': 'gpt-35-turbo-16k', 'system_fingerprint': None, 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {}}], 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}, id='run-a817e408-7894-49f1-b661-0fdee35b1a78-0')